## Day1
https://adventofcode.com/2021/day/1

In [ ]:
with open('inputs_2021/day1.txt', 'r') as file:
    depths = [int(row.strip()) for row in file]

measurement_increases = sum(
    current > previous for previous, current in zip(depths, depths[1:])
)

print(measurement_increases)

1711


In [22]:
sliding_windows = [
    sum(depths[i:i+3]) for i in range(len(depths) - 2)
]

sliding_windows_increaces = sum(
    current > previous for previous, current in zip(sliding_windows, sliding_windows[1:])
)
print(sliding_windows_increaces)

1743


## Day2
https://adventofcode.com/2021/day/2

In [38]:
position_changes = {}

with open('inputs_2021/day2.txt', 'r') as file:
    for row in file:
        key, value = row.strip().split()
        position_changes[key] = position_changes.get(key, 0) + int(value)

print(position_changes['forward'] * (position_changes['down'] - position_changes['up']))

2187380


In [ ]:
from collections import defaultdict

positions = defaultdict(int)

with open('inputs_2021/day2.txt', 'r') as file:
    for row in file:
        key, value = row.strip().split()
        value = int(value)
        aim = 0

        match key:
            case 'down':
                aim += value
            case 'up':
                aim -= value
            case  'forward':
                positions['forward'] += value
                positions['down'] += aim * value

print(positions)
print(positions['forward'] * positions['down'])

defaultdict(<class 'int'>, {'forward': 1790, 'down': 1165563, 'aim': 1222})
2086357770


## Day3
https://adventofcode.com/2021/day/3

In [116]:
from collections import defaultdict

positions = defaultdict(int)

with open('inputs_2021/day3.txt', 'r') as file:
    file_len = 0
    for line in file:
        file_len += 1
        for i, bit in enumerate(line.strip()):
            positions[i] += int(bit)

threshold = file_len / 2

gamma_rate, epsilon_rate = zip(*[
    ('1', '0') if val >= threshold else ('0', '1') 
    for val in positions.values()
])

print(int(''.join(gamma_rate), 2) * int(''.join(epsilon_rate), 2))

693486


In [ ]:
def parse_input(filepath: str) -> list[list[int]]:
    """Reads the input file and returns a list of lists of integers."""
    with open(filepath, 'r') as file:
        return [list(map(int, line.strip())) for line in file]

def find_rating(data: list[list[int]], is_gamma: bool) -> int:
    bit_length = len(data[0])

    for i in range(bit_length):
        if len(data) == 1:
            break
        ones = sum(row[i] for row in data)
        zeroes = len(data) - ones
        if is_gamma:
            keep_bit = 1 if ones >= zeroes else 0
        else:
            keep_bit = 0 if ones >= zeroes else 1
        data = [row for row in data if row[i] == keep_bit]

    return int(''.join(map(str, data[0])), 2)

data = parse_input('inputs_2021/day3.txt')

oxygen_rating = find_rating(data, True)
co2_rating = find_rating(data, False)

# Multiply and print
print(oxygen_rating * co2_rating)

3379326


## Day4
https://adventofcode.com/2021/day/4

In [87]:
def parse_input(filepath: str) -> list[list[int]]:
    """Reads the input file and returns a list of lists of integers."""
    with open(filepath) as f:
        parts = f.read().strip().split('\n\n')

    inputs = list(map(int, parts[0].split(',')))

    boards = [
        [list(map(int, row.split())) for row in block.splitlines()]
        for block in parts[1:]
    ]
    return inputs, boards
    

inputs, boards_raw = parse_input('inputs_2021/day4.txt')


In [ ]:
import numpy as np

class BingoBoard:
    def __init__(self, grid):
        self.numbers = np.array(grid)
        self.marked = np.zeros((5, 5), dtype=bool)

    def mark(self, number):
        self.marked |= (self.numbers == number)

    @property
    def is_winning(self):
        return np.any(np.all(self.marked, axis=0)) or np.any(np.all(self.marked, axis=1))

    @property
    def unmarked_sum(self):
        return np.sum(self.numbers[~self.marked])

    def score(self, last_number):
        return self.unmarked_sum * last_number

boards = [BingoBoard(grid) for grid in boards_raw]

for number in inputs:
    for board in boards:
        board.mark(number)
        if board.is_winning:
            print("Winner!")
            print("Score:", board.score(number))
            break
    else:
        continue
    break

Winner!
Score: 97565


In [ ]:
def find_last_winning_board(boards_raw, inputs) -> int:

    remaining_boards = [BingoBoard(grid) for grid in boards_raw]

    for number in inputs:
        newly_won = []

        for board in remaining_boards:
            board.mark(number)
            if board.is_winning:
                if len(remaining_boards) == 1:
                    print("Looser found!")
                    return board.score(number)
                else:
                    newly_won.append(board)

        remaining_boards = [b for b in remaining_boards if b not in newly_won]

print(f"Looser's score is {find_last_winning_board(boards_raw, inputs)}")

Looser found!
Looser's score is 1827


## DAY5

In [27]:
import numpy as np

def parse_lines(filename):
    with open(filename) as f:
        for line in f:
            x1, y1, x2, y2 = map(int, line.replace(' -> ', ',').split(','))
            yield x1, y1, x2, y2

def get_grid(lines, diagonals=False, size=1000):
    grid = np.zeros((size, size), dtype=int)
    for x1, y1, x2, y2 in lines:
        dx = np.sign(x2 - x1)
        dy = np.sign(y2 - y1)

        length = max(abs(x2 - x1), abs(y2 - y1))
        if not diagonals and dx != 0 and dy != 0:
            continue

        for i in range(length + 1):
            grid[y1 + i * dy, x1 + i * dx] += 1
    return grid


In [28]:
lines = list(parse_lines('inputs_2021/day5.txt'))
grid1 = get_grid(lines, diagonals=False)
grid2 = get_grid(lines, diagonals=True)

print("Part 1:", np.sum(grid1 >= 2))
print("Part 2:", np.sum(grid2 >= 2))

Part 1: 5092
Part 2: 20484


## DAY 6

In [1]:
from collections import Counter

# Parse input
with open("inputs_2021/day6.txt") as f:
    lanternfishes = list(map(int, f.readline().strip().split(",")))

def fish_simulation(fish_counts: dict, days_to_simulate: int):

    fish_dict =  fish_counts.copy()
    for _ in range(days_to_simulate):
        next_day = Counter()
        for timer, count in fish_dict.items():
            if timer == 0:
                next_day[6] += count
                next_day[8] += count
            else:
                next_day[timer - 1] += count
        fish_dict = next_day

    # Final count
    return sum(fish_dict.values())


In [2]:
fish_counts = Counter(lanternfishes)

fish_after_80_days = fish_simulation(fish_counts, 80)
fish_after_256_days = fish_simulation(fish_counts, 256)


print(fish_after_80_days)
print(fish_after_256_days)

345387
1574445493136


## DAY 7

In [38]:
from collections import Counter

with open('inputs_2021/day7.txt') as file:
    positions = Counter(map(int, file.readline().strip().split(",")))

low_consumption_costs = {
    i: sum(abs(i - key) * value for key, value in positions.items())
    for i in range(min(positions), max(positions) + 1)
}

print(min(low_consumption_costs.values()))


364898


In [39]:
def triangular_number(n):
    return n * (n + 1) // 2

high_consumption_costs = {
    i: sum(triangular_number(abs(i - key)) * value for key, value in positions.items())
    for i in range(min(positions), max(positions) + 1)
}

print(min(high_consumption_costs.values()))

104149091


## DAY 8 

In [11]:
with open('inputs_2021/day8.txt') as file:
    signals = [
        line.split("|")[1].strip().split()
        for line in file
    ]

uniques = sum(
        1
        for segments in signals
        for segment in segments
        if len(segment) in [2, 3, 4, 7]
    )

print(uniques)

318


In [82]:
from dataclasses import dataclass

@dataclass
class SignalEntry:
    patterns: list[frozenset]
    outputs: list[frozenset]
    

def decode(signal: SignalEntry) -> int:
    # Group patterns by their length for fast access
    patterns_by_length = {length: [] for length in range(2, 8)}
    for p in signal.patterns:
        patterns_by_length[len(p)].append(p)

    one = patterns_by_length[2][0]
    four = patterns_by_length[4][0]
    seven = patterns_by_length[3][0]
    eight = patterns_by_length[7][0]

    # Find three: 5-segment pattern containing one
    three = next(p for p in patterns_by_length[5] if one <= p)
    # Find nine: 6-segment pattern containing four
    nine = next(p for p in patterns_by_length[6] if four <= p)
    # Find five: 5-segment pattern containing (four - one) and not three
    bd = four - one
    five = next(p for p in patterns_by_length[5] if bd <= p and p != three)
    # Find two: the remaining 5-segment pattern
    two = next(p for p in patterns_by_length[5] if p not in {three, five})
    # Find six: 6-segment pattern that doesn't fully contain one and is not nine
    six = next(p for p in patterns_by_length[6] if not (one <= p) and p != nine)
    # Find zero: the remaining 6-segment pattern
    zero = next(p for p in patterns_by_length[6] if p not in {six, nine})
    # Build the decoder
    decoder = {
        zero: 0,
        one: 1,
        two: 2,
        three: 3,
        four: 4,
        five: 5,
        six: 6,
        seven: 7,
        eight: 8,
        nine: 9,
    }

    # Decode the output
    output_value = int("".join(str(decoder[o]) for o in signal.outputs))
    return output_value

signals = []
with open('inputs_2021/day8.txt') as file:
    for line in file:
        left, right = line.strip().split("|")

        outputs = [frozenset(segment) for segment in right.strip().split()]
        patterns = [frozenset(segment) for segment in left.strip().split()]

        signals.append(SignalEntry(patterns=patterns, outputs=outputs))

total = sum(decode(signal) for signal in signals)
print(total)

996280


In [ ]:
from itertools import permutations

# Canonical digits - real wiring
REAL_DIGITS = {
    frozenset('abcefg'): 0,
    frozenset('cf'): 1,
    frozenset('acdeg'): 2,
    frozenset('acdfg'): 3,
    frozenset('bcdf'): 4,
    frozenset('abdfg'): 5,
    frozenset('abdefg'): 6,
    frozenset('acf'): 7,
    frozenset('abcdefg'): 8,
    frozenset('abcdfg'): 9,
}

def decode(signal: SignalEntry) -> int:
    scrambled_patterns = signal.patterns

    for perm in permutations('abcdefg'):
        mapping = {scrambled: real for scrambled, real in zip(perm, 'abcdefg')}

        # Remap all scrambled patterns
        remapped = [
            frozenset(mapping[char] for char in pattern)
            for pattern in scrambled_patterns
        ]

        if all(pattern in REAL_DIGITS for pattern in remapped):
            # Found correct mapping
            pattern_to_digit = {pattern: digit for pattern, digit in REAL_DIGITS.items()}

            decoded_number = int(
                "".join(
                    str(pattern_to_digit[frozenset(mapping[c] for c in output)])
                    for output in signal.outputs
                )
            )
            return decoded_number

    raise ValueError("No valid mapping found!")

signals = []
with open('inputs_2021/day8.txt') as file:
    for line in file:
        left, right = line.strip().split("|")

        outputs = [frozenset(segment) for segment in right.strip().split()]
        patterns = [frozenset(segment) for segment in left.strip().split()]

        signals.append(SignalEntry(patterns=patterns, outputs=outputs))

total = sum(decode(signal) for signal in signals)
print(total)

996280
